# PWV02 : Explore Statistical Uncertainies from fits colored by SED type

- author Sylvie Dagoret-Campagne
- creation date 2025-10-29 : version v10
- affiliation : IJCLab
- last update : 2025-10-31 : Plot colors related to Spectral Type
- Kernel @usdf **w_2025_42*
- Home emac : base (conda)
- laptop : conda_py313

**Goal** : Show Night variations of PWV wrt date and Time. Fit a straight line.

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import os

In [ ]:
# where are stored the figures
pathfigs = "figs_PWV02_StatErrors"
prefix = "pwv02-staterr"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (16,8)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams["legend.fontsize"] = "xx-large"

import scipy
from scipy.optimize import curve_fit,least_squares


# new color correction model
import pickle
from scipy.interpolate import RegularGridInterpolator

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time

from datetime import datetime, timezone

In [ ]:
# Remove to run faster the notebook
#import ipywidgets as widgets
#%matplotlib widget

In [ ]:
from PWV00_parameters import *

In [ ]:
DumpConfig()

In [ ]:
from importlib.metadata import version

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
np.__version__

In [ ]:
pd.__version__

### Configuration

In [ ]:
def q50(x):
    return np.quantile(x,0.5)

# 90th Percentile
def q90(x):
    return np.quantile(x,0.9)

def q25(x):
    return np.quantile(x,0.25)

def q75(x):
    return np.quantile(x,0.75)

def iqr(x):
    irq =  q75(x)-q25(x)
    return irq

#https://en.wikipedia.org/wiki/Interquartile_range    
#SD = IQR/1.35. for a normal distribution
def std_iqr(x):
    return iqr(x)/1.349
    
#df.groupby("AGGREGATE").quantile([0, 0.25, 0.5, 0.75, 0.95, 1])
#df.groupby("AGGREGATE").agg(("YOUR_COL_NAME", lambda x: x.quantile(0.5))

In [ ]:
def delta_angle_deg(a1, a2):
    """
    Différence d'angle (en degrés) ramenée à l'intervalle [-180, 180].
    """
    diff = (a1 - a2 + 180) % 360 - 180
    return diff

In [ ]:
def convertNumToDatestr(num):
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return pd.to_datetime(datestr)

In [ ]:
PWVMIN = 0.
PWVMAX = 15.

In [ ]:
FLAG_WITHCOLLIMATOR = True
DATE_WITHCOLLIMATOR = 20230930
datetime_WITHCOLLIMATOR = convertNumToDatestr(DATE_WITHCOLLIMATOR)
datetime_WITHCOLLIMATOR = pd.to_datetime("2023-09-30 00:00:00.0+0000")
datetime_WITHCOLLIMATOR

## Initialisation

### Read the file
- `atmfilename` is defined in `PW00_parameters.py` 

In [ ]:
version_run = "run_v10"
FLAG_REPO_EMBARGO = map_run_butler_embargo[version_run]

In [ ]:
atmfilename = mergedextractedfilesdict[version_run]
tag = legendtag[version_run] 

In [ ]:
the_collection = butlerusercollectiondict[version_run] 
the_suptitle = butlerusercollectiondict[version_run] 

In [ ]:
specdata = np.load(atmfilename,allow_pickle=True)

In [ ]:
df_spec = pd.DataFrame(specdata)

In [ ]:
#if FLAG_WITHCOLLIMATOR:
#    df_spec = df_spec[df_spec["nightObs"]> DATE_WITHCOLLIMATOR]    

In [ ]:
# add time for plotting and sort 
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"],utc=True)
#df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"]).dt.tz_convert("UTC")
df_spec = df_spec.sort_values(by='Time')

In [ ]:
df_spec["nightObs"] = df_spec.apply(lambda x: x['id']//100_000, axis=1)

In [ ]:
#df_spec = df_spec[df_spec["nightObs"]> 20250000]

In [ ]:
DT = pd.Timedelta(minutes=7*24*60)
TMIN  = df_spec["Time"].min()-DT
TMAX  = df_spec["Time"].max()+DT

In [ ]:
df_spec["seq_num"]  = df_spec["id"] % 100_000

In [ ]:
df_spec[["id","FILTER"]]

In [ ]:
df_spec["FILTER"].unique()

In [ ]:
print(list(df_spec.columns))

### Check Filters

In [ ]:
# conversion en datetime

plt.figure(figsize=(20,8))
sns.scatterplot(
    data=df_spec, 
    x="Time",       # abscisse en datetime
    y="seq_num",    # ou ra, dec, etc.
    hue="FILTER", 
    palette="tab10",
    s=100,
    edgecolor="black",
    linewidth=0.2
)

plt.title(f"Auxtel Holo observations wrt date and filter type, {tag}")
plt.xlabel("Date of observation")
plt.ylabel("Seq Num")
plt.xticks(rotation=45)  # lisibilité des dates
#plt.legend(loc="upper left",ncol=8)
plt.legend(bbox_to_anchor=(1.01, 1.05),ncols=1)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(20,8))
df_spec["FILTER_seq"] = df_spec["FILTER"].astype(str) + "_" + df_spec["seq_num"].astype(str)
sns.stripplot(
    data=df_spec,
    x="Time",
    y="FILTER",
    hue="FILTER",
    palette="Set1",
    size=10,         # taille des points
    jitter=True,    # évite que les points se chevauchent
    alpha=1.0,
    edgecolor="black",
    linewidth=0.1
)
plt.title(f"Auxtel Holo observations wrt date and filter type,  {tag}")
plt.xlabel("Time")
plt.ylabel("Filter")
plt.xticks(rotation=45)
plt.tight_layout()
plt.grid()
plt.show()


## Suppress Blue filters

In [ ]:
if FLAG_PWVFILTERS: 
    df_spec = df_spec[df_spec["FILTER"].isin(PWV_FILTER_LIST) ]
    #df_spec = df_spec[df_spec["FILTER"].isin(PWV_FILTEROG550_LIST) ]

In [ ]:

# Compter le nombre d’entrées par nightObs et FILTER
counts = df_spec.groupby(["nightObs", "FILTER"]).size().unstack(fill_value=0)

# Plot en barres empilées
counts.plot(kind="bar", stacked=False, figsize=(18,6))

plt.ylabel("Nombre d'entrées")
plt.xlabel("nightObs")
plt.title(f"Nombre d'entrées par FILTER et par nightObs, {tag}")
plt.legend(title="FILTER")
plt.tight_layout()
plt.show()


### Visualize Selection cuts

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(18,3))
ax1,ax2,ax3  = axs.flatten()
df_spec.hist("D_CCD [mm]_x",ax=ax1,bins=50,range=(DCCDMINFIG,DCCDMAXFIG),facecolor="b")
ax1.axvline(DCCDMINCUT,ls="-.",c="k")
ax1.axvline(DCCDMAXCUT,ls="-.",c="k")

df_spec.hist("CHI2_FIT",ax=ax2,bins=50,range=(0,500),facecolor="b")
#ax2.set_yscale("log")
ax2.axvline(CHI2CUT,ls="-.",c="k")


df_spec.hist("EXPTIME",ax=ax3,bins=20,facecolor="b")
ax3.axvline(EXPTIMECUT,ls="-.",c="k") 
plt.suptitle(tag)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

filters = df_spec["FILTER"].unique()

fig, axs = plt.subplots(len(filters), 3, figsize=(18, 3*len(filters)))

for i, f in enumerate(filters):
    subdf = df_spec[df_spec["FILTER"] == f]  # sélectionne uniquement ce filtre
    
    ax1, ax2, ax3 = axs[i] if len(filters) > 1 else axs  # gestion si 1 seul filtre
    
    subdf.hist("D_CCD [mm]_x", ax=ax1, bins=50,
               range=(DCCDMINFIG, DCCDMAXFIG), facecolor="b")
    ax1.axvline(DCCDMINCUT, ls="-.", c="k")
    ax1.axvline(DCCDMAXCUT, ls="-.", c="k")
    ax1.set_title(f"{f} – D_CCD [mm]_x")

    subdf.hist("CHI2_FIT", ax=ax2, bins=50,range=(0,300) ,facecolor="b")
    #ax2.set_yscale("log")
    ax2.axvline(CHI2CUT, ls="-.", c="k")
    ax2.set_title(f"{f} – CHI2_FIT")
   

    subdf.hist("EXPTIME", ax=ax3, bins=20, facecolor="b")
    ax3.axvline(EXPTIMECUT, ls="-.", c="k")
    ax3.set_title(f"{f} – EXPTIME")

plt.suptitle(f" Quality selection cut , {tag}")
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")
fig, axs = plt.subplots(2, 1, figsize=(18, 12))
ax1, ax2 = axs

# Scatter avec couleur selon FILTER
sc = ax1.scatter(
    df_spec["Time"],
    df_spec["D_CCD [mm]_x"],
    c=df_spec["FILTER"].astype("category").cat.codes,  # codes numériques
    cmap="Set1",  # palette discrète
    marker="+",lw=5,alpha=0.5
)
ax1.set_ylim(DCCDMINFIG,DCCDMAXFIG)
ax1.axhline(DCCDMINCUT,ls="-.",c="k")
ax1.axhline(DCCDMAXCUT,ls="-.",c="k")

# Légende avec les filtres
handles, labels = sc.legend_elements(prop="colors", alpha=0.6)
ax1.legend(handles, df_spec["FILTER"].unique(), title="FILTER",ncols=8)
#ax1.legend(bbox_to_anchor=(1.01, 1.05),ncols=1,title="FILTER")

plt.setp(ax1.get_xticklabels(), rotation=45, ha="right")
ax1.set_ylabel("D_CCD [mm]_x")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax1.set_title("DCCD vs time")
ax1.set_ylim(DCCDMINFIG,DCCDMAXFIG)
ax1.axhline(DCCDMINCUT,ls="-.",c="k")
ax1.axhline(DCCDMAXCUT,ls="-.",c="k")

sc = ax2.scatter(
    df_spec["Time"],
    df_spec["CHI2_FIT"],
    c=df_spec["FILTER"].astype("category").cat.codes,  # codes numériques
    cmap="Set1",  # palette discrète
    marker="+",lw=5,alpha=0.5
)
ax2.set_yscale("log")
ax2.axhline(CHI2CUT,ls="-.",c="k")

handles, labels = sc.legend_elements(prop="colors", alpha=0.6)
ax2.legend(handles, df_spec["FILTER"].unique(), title="FILTER",ncols=8)
#x2.legend(bbox_to_anchor=(1.01, 1.05),ncols=1,title="FILTER")

plt.setp(ax2.get_xticklabels(), rotation=45, ha="right")
ax2.set_ylabel("CHI2_FIT")
ax2.set_xlabel("time")
ax2.xaxis.set_major_formatter(date_form)
ax2.set_title("CHI2_FIT")

plt.suptitle(tag)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

date_form = DateFormatter("%y-%m-%d")

filters = df_spec["FILTER"].unique()

fig, axs = plt.subplots(len(filters), 2, figsize=(18, 6*len(filters)))
if len(filters) == 1:
    axs = [axs]  # uniformiser si un seul filtre

for i, f in enumerate(filters):
    subdf = df_spec[df_spec["FILTER"] == f]
    ax1, ax2 = axs[i]

    # --- DCCD vs time ---
    ax1.scatter(
        subdf["Time"],
        subdf["D_CCD [mm]_x"],
        marker="+", lw=5, alpha=0.5, color="b"
    )
    ax1.set_ylim(DCCDMINFIG, DCCDMAXFIG)
    ax1.axhline(DCCDMINCUT, ls="-.", c="k")
    ax1.axhline(DCCDMAXCUT, ls="-.", c="k")
    ax1.set_ylabel("D_CCD [mm]_x")
    ax1.set_xlabel("time")
    ax1.xaxis.set_major_formatter(date_form)
    ax1.set_title(f"{f} – DCCD vs time")
    plt.setp(ax1.get_xticklabels(), rotation=45, ha="right")

    # --- CHI2 vs time ---
    ax2.scatter(
        subdf["Time"],
        subdf["CHI2_FIT"],
        marker="+", lw=5, alpha=0.5, color="b"
    )
    ax2.set_yscale("log")
    ax2.axhline(CHI2CUT, ls="-.", c="k")
    ax2.set_ylabel("CHI2_FIT")
    ax2.set_xlabel("time")
    ax2.xaxis.set_major_formatter(date_form)
    ax2.set_title(f"{f} – CHI2_FIT")
    plt.setp(ax2.get_xticklabels(), rotation=45, ha="right")

plt.suptitle(tag)
plt.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

for filt, group in df_spec.groupby("FILTER"):
    ax.scatter(
        group["AIRMASS"],
        group["CHI2_FIT"],
        label=filt,
        alpha=0.6
    )

ax.set_xlabel("Airmass")
ax.set_ylabel("CHI2_FIT")
ax.set_yscale("log")   # utile si chi2 très dispersé
ax.legend(title="FILTER")
ax.grid(True)
plt.tight_layout()
plt.show()

### Atmospheric parameters distribution before selection

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(16,10))
ax1,ax2,ax3,ax4 = axs.flatten()
    
df_spec.hist("PWV [mm]_x",ax=ax1,bins=50,facecolor="b")
df_spec.hist("PWV [mm]_y",ax=ax2,bins=50,facecolor="b")
df_spec.hist("ozone [db]_x",ax=ax3,bins=50,facecolor="r")
df_spec.hist("ozone [db]_y",ax=ax4,bins=50,facecolor="r")
plt.suptitle(f"Atmospheric parameters before selection, {tag}")
plt.tight_layout()
plt.show()

### Target used

In [ ]:
ListOfTargets = df_spec["TARGET"].unique()
print(ListOfTargets)

In [ ]:
plt.figure(figsize=(20,12))
df_spec["TARGET_seq"] = df_spec["TARGET"].astype(str) + "_" + df_spec["seq_num"].astype(str)
sns.stripplot(
    data=df_spec,
    x="Time",
    y="TARGET",
    hue="TARGET",
    palette="Set1",
    size=10,         # taille des points
    jitter=True,    # évite que les points se chevauchent
    alpha=1.0,
    edgecolor="black",
    linewidth=0.1
)
plt.title(f"Auxtel Holo observations wrt date and target {tag}")
plt.xlabel("Time")
plt.ylabel("Target")
plt.xticks(rotation=45)
plt.tight_layout()
plt.grid()
plt.show()

### Is the star Faint or bright

In [ ]:
def IsFaint(row):
    List_Of_Faint_targets = ['Feige110','HD074000','HD115169','HD031128','HD200654','HD167060','HD009051','HD142331','HD160617','HD111980']
    List_Of_faint_selected = List_Of_Faint_targets[:4]
    if row["TARGET"] in List_Of_faint_selected:
        return True
    else:
        return False

In [ ]:
df_spec["isFaint"] = df_spec.apply(IsFaint,axis=1)

### Collimator

In [ ]:
if FLAG_WITHCOLLIMATOR:
    df_spec = df_spec[df_spec["nightObs"]> DATE_WITHCOLLIMATOR ]

## DO NOT Apply or not correction on errors related to PWV repeatability

In [ ]:
# Take into account Photometric Repeatability
#if FLAG_CORRECTFOR_PWV_REPEAT:
#    if FLAG_CORRECTFOR_PWV_REPEAT_RATIO:
#        df_spec["PWV [mm]_err_x"] =  df_spec["PWV [mm]_err_x"] * FACTORERR_PWV_REPEAT
#    else:
#        df_spec["PWV [mm]_err_x"] =  np.sqrt(df_spec["PWV [mm]_err_x"]**2  +   SIGMA_PWV_REPEAT**2)
    

### Series on spec

In [ ]:
ser_spec_size = df_spec.groupby(["nightObs"]).size()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,5))
ax.hist(ser_spec_size.values,bins=50,facecolor="b")
ax.set_title(f"nb obs per night {tag}")
ax.set_xlabel("Nobs/night")
plt.show()

##### Make 3 series

In [ ]:
ser_CHI2_FIT = df_spec[["CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])
ser_PWV = df_spec[["PWV [mm]_x","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])
ser_PWV_CHI2_FIT = df_spec[["PWV [mm]_x","CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])

### Plot PWV and Chi2 from series before any selection

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV.unstack()["PWV [mm]_x"]["count"].plot(kind='bar', ax=ax,subplots=False, rot=90,figsize=(18,4),facecolor="b",grid=True,title=f"Number of measurements per night, {tag}")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV.unstack()["PWV [mm]_x"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title=f"Mean PWV per night , {tag}")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV.unstack()["PWV [mm]_x"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title=f"Median PWV per night ,{tag}")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV.unstack()["PWV [mm]_x"]["std"].plot(kind='bar', subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title=f"STD variation for PWV per night , {tag}")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["count"].plot(kind='bar', ax=ax,subplots=False, rot=90,figsize=(18,4),facecolor="r",grid=True,title=f"Number of measurements per night, {tag}")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title=f"Mean CHI2 per night, {tag}")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title=f"Median CHI2 per night , {tag}")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["std"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title=f"STD variation CHI2 per night , {tag}")
plt.tight_layout()

#### Add aggregate data added to pandas dataframe

In [ ]:
def FillAgreggates(row):
    the_nightObs = row["nightObs"]
    df_night = ser_PWV_CHI2_FIT.loc[the_nightObs,:].unstack()
    count = df_night.loc["PWV [mm]_x","count"]
    pwvmin = df_night.loc["PWV [mm]_x","min"]
    pwvmax = df_night.loc["PWV [mm]_x","max"]
    pwvmean = df_night.loc["PWV [mm]_x","mean"]
    pwvmedian = df_night.loc["PWV [mm]_x","median"]
    pwvstd = df_night.loc["PWV [mm]_x","std"]
    chi2min = df_night.loc["CHI2_FIT","min"]
    chi2max = df_night.loc["CHI2_FIT","max"]
    chi2mean = df_night.loc["CHI2_FIT","mean"]
    chi2median = df_night.loc["CHI2_FIT","median"]
    chi2std = df_night.loc["CHI2_FIT","std"]
    
    d = {"_count":count,"_pwvmin":pwvmin,"_pwvmax":pwvmax,"_pwvmean":pwvmean,"_pwvmedian":pwvmedian,"_pwvstd":pwvstd,
        "_chi2min":chi2min,"_chi2max":chi2max,"_chi2mean":chi2mean,"_chi2median":chi2median,"_chi2std":chi2std}
    
    ser = pd.Series(data=d)

    return ser

In [ ]:
row = df_spec.iloc[0]

In [ ]:
row

In [ ]:
FillAgreggates(row)

#### join dataframe + aggregates

In [ ]:
df_spec = df_spec.join(df_spec.apply(FillAgreggates,axis=1,result_type="expand"))

## Apply Quality selection

In [ ]:
DT = pd.Timedelta(minutes=7*24*60)
TMIN  = df_spec["Time"].min()-DT
TMAX  = df_spec["Time"].max()+DT

## Compute night boundaries

In [ ]:
def GetNightBoundariesDict(df_spec):
    """
    input:
      df_spec the dataframe for spectroscopy summary results
    output:
      the dict of night boudaries
    """
    
    Dt = pd.Timedelta(minutes=30)
    d = {}
    list_of_nightobs = df_spec["nightObs"].unique()
    for nightobs in list_of_nightobs:
        sel_flag = df_spec["nightObs"]== nightobs
        df_night = df_spec[sel_flag]
        tmin = df_night["Time"].min()-Dt
        tmax = df_night["Time"].max()+Dt
        d[nightobs] = (tmin,tmax)
    return d

In [ ]:
dn = GetNightBoundariesDict(df_spec)

## Compute night midnights

In [ ]:
def GetNightMidnightsDict(df_spec):
    """
    input:
      df_spec the dataframe for spectroscopy summary results
    output:
      the dict of midnights
    """
    
    Dt = pd.Timedelta(minutes=30)
    d = {}
    list_of_nightobs = df_spec["nightObs"].unique()
    for nightobs in list_of_nightobs:
        #sel_flag = df_spec["nightObs"]== nightobs
        #df_night = df_spec[sel_flag]
        #tmin = df_night["Time"].min()-Dt
        #tmax = df_night["Time"].max()+Dt
        nightstr = datetime.strptime(str(nightobs), "%Y%m%d")
        midnight = get_astronomical_midnight(site_lsst, nightstr.date())
        d[nightobs] = midnight
        
    return d

In [ ]:
dnidnights = GetNightMidnightsDict(df_spec)

## Plot all data

In [ ]:
list_all_filts = df_spec["FILTER"].unique()
list_all_filts= sorted(list_all_filts )
colors = {filt: col for filt, col in zip(list_all_filts, ["r","b"])}

In [ ]:
colors = {
    'empty': "blue", 
    'FELH0600': "purple",
    'OG550_65mm_1':"red",
}

In [ ]:
date_form = DateFormatter("%y-%m-%d")

fig,axs = plt.subplots(1,1,figsize=(18,8))
ax  = axs
leg=ax.get_legend()

#df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg)

for filt, group in df_spec.groupby("FILTER"):
    ax.errorbar(
                    group["Time"],
                    group["PWV [mm]_x"],
                    yerr= group["PWV [mm]_err_x"],
                    fmt="o",
                    label=filt,
                    color=colors[filt],
                    ecolor="k",
                    capsize=1,
                    markersize=5
                    )


ax.set_ylabel("PWV [mm]_x")

ax.set_xlabel("Time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title(f"Precipitable water vapor measured by holo vs time (before selection cut) , {tag}")
ax.tick_params(axis="x", rotation=45)
ax.legend(loc="upper right")
ax.set_ylim(PWVMIN,PWVMAX)

for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.05)

for key, midn in dnidnights.items():
    ax.axvline( midn ,color="purple",ls=":",alpha=0.5)

if not FLAG_WITHCOLLIMATOR:
    ax.axvspan(TMIN,datetime_WITHCOLLIMATOR, color='yellow', alpha=0.1)

plt.tight_layout()

figname =f"{pathfigs}/{prefix}_pwv_allpoints_allnights_noqualcuts"+figtype
plt.savefig(figname)
plt.show()


## Apply Quality selection cuts

In [ ]:
cut = getSelectionCut(df_spec)

In [ ]:
df_spec_sel = df_spec[cut].drop(labels=['_count', '_pwvmin', '_pwvmax', '_pwvmean', '_pwvmedian', '_pwvstd','_chi2min', '_chi2max', '_chi2mean', '_chi2median', '_chi2std'],axis=1)

In [ ]:
df_spec_sel.reset_index(drop=True,inplace=True)

### Atmospheric parameters **after** selection

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(16,10))
ax1,ax2,ax3,ax4 = axs.flatten()
    
df_spec_sel.hist("PWV [mm]_x",ax=ax1,bins=50,facecolor="b")
df_spec_sel.hist("PWV [mm]_y",ax=ax2,bins=50,facecolor="b")
df_spec_sel.hist("ozone [db]_x",ax=ax3,bins=50,facecolor="r")
df_spec_sel.hist("ozone [db]_y",ax=ax4,bins=50,facecolor="r")
plt.suptitle(f"Atmospheric parameters after selection, {tag}")
plt.tight_layout()
plt.show()

## Compute per-night aggregates

### Compute series per night

In [ ]:
ser_PWV_CHI2_FIT_sel = df_spec_sel[["PWV [mm]_x","CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])

In [ ]:
def FillAgreggatesSel(row):
    the_nightObs = row["nightObs"]
    df_night = ser_PWV_CHI2_FIT_sel.loc[the_nightObs,:].unstack()
    count = df_night.loc["PWV [mm]_x","count"]
    pwvmin = df_night.loc["PWV [mm]_x","min"]
    pwvmax = df_night.loc["PWV [mm]_x","max"]
    pwvmean = df_night.loc["PWV [mm]_x","mean"]
    pwvmedian = df_night.loc["PWV [mm]_x","median"]
    pwvstd = df_night.loc["PWV [mm]_x","std"]
    chi2min = df_night.loc["CHI2_FIT","min"]
    chi2max = df_night.loc["CHI2_FIT","max"]
    chi2mean = df_night.loc["CHI2_FIT","mean"]
    chi2median = df_night.loc["CHI2_FIT","median"]
    chi2std = df_night.loc["CHI2_FIT","std"]
    
    d = {"_count":count,"_pwvmin":pwvmin,"_pwvmax":pwvmax,"_pwvmean":pwvmean,"_pwvmedian":pwvmedian,"_pwvstd":pwvstd,
        "_chi2min":chi2min,"_chi2max":chi2max,"_chi2mean":chi2mean,"_chi2median":chi2median,"_chi2std":chi2std}
    
    ser = pd.Series(data=d)

    return ser
    

In [ ]:
df_spec_sel = df_spec_sel.join(df_spec_sel.apply(FillAgreggatesSel,axis=1,result_type="expand"))

## Recompute night boundaries

In [ ]:
dn = GetNightBoundariesDict(df_spec_sel)

## Plot all data

In [ ]:
list_all_filts = df_spec_sel["FILTER"].unique()
list_all_filts= sorted(list_all_filts )
colors = {filt: col for filt, col in zip(list_all_filts, ["r","b"])}
colors = {
    'empty': "blue", 
    'FELH0600': "purple",
    'OG550_65mm_1':"red",
}

In [ ]:
from matplotlib.dates import DateFormatter
#date_form = DateFormatter("%y-%m-%dT%H:%M")
date_form = DateFormatter("%y-%m-%d")


fig,axs = plt.subplots(1,1,figsize=(18,8))
ax  = axs
leg=ax.get_legend()

#df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg)

for filt, group in df_spec_sel.groupby("FILTER"):
    ax.errorbar(
                    group["Time"],
                    group["PWV [mm]_x"],
                    yerr= group["PWV [mm]_err_x"],
                    fmt="o",
                    label=filt,
                    color=colors[filt],
                    ecolor="k",
                    capsize=1,
                    markersize=5
                    )


ax.set_ylabel("PWV [mm]_x")

ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title(f"Precipitable water vapor measured by holo vs time (after selection cut) , {tag}")
ax.tick_params(axis="x", rotation=45)
ax.legend(loc="upper right")
ax.grid()

for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.05)

for key, midn in dnidnights.items():
    ax.axvline( midn ,color="purple",ls=":",alpha=0.5)

ax.set_ylim(PWVMIN,PWVMAX)

if not FLAG_WITHCOLLIMATOR:
    ax.axvspan(TMIN,datetime_WITHCOLLIMATOR, color='yellow', alpha=0.1)

plt.tight_layout()
figname =f"{pathfigs}/{prefix}_pwv_allpoints_allnights_withqualcuts"+figtype
plt.savefig(figname)
plt.ylim(0.,20.)
plt.show()

## Plot series on selected data

In [ ]:
ser_CHI2_FIT_sel = df_spec_sel[["CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])
ser_PWV_sel = df_spec_sel[["PWV [mm]_x","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV_sel.unstack()["PWV [mm]_x"]["count"].plot(kind='bar', ax=ax,subplots=False, rot=90,figsize=(18,4),facecolor="b",grid=True,title=f"Number of measurements per night after selection {tag}")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV_sel.unstack()["PWV [mm]_x"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title=f"Mean PWV per night after selection {tag}")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV_sel.unstack()["PWV [mm]_x"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title=f"Median PWV per night after selection {tag}")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV_sel.unstack()["PWV [mm]_x"]["std"].plot(kind='bar', subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title=f"STD variation for PWV per night after selection {tag}")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT_sel.unstack()["CHI2_FIT"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title=f"Mean CHI2 per night after selection {tag}")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT_sel.unstack()["CHI2_FIT"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title=f"Median CHI2 per night after selection {tag}")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT_sel.unstack()["CHI2_FIT"]["std"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title=f"STD variation CHI2 per night after selection {tag}")
plt.tight_layout()

## Special study on statistical error

#### Load magnitudes from external file
- the magnitudes have been computed in another notebooks in `../2025-10-29-TOOLS`

In [ ]:
targets_mag_files = "../2025-10-29-TOOLS/data/targets_magnitudes.csv"
df_targets_mag = pd.read_csv(targets_mag_files,index_col=0)      
df_targets_mag = df_targets_mag.sort_values(by="y")

#### palette with SED type

In [ ]:
df_col = df_targets_mag.copy()
df_col = df_col.sort_values(by="B_V")

SpT = df_col["Sp_T"].values
unique_types = list(dict.fromkeys(SpT))  # garde l'ordre d'apparition
N_types = len(unique_types)


# Associe chaque type spectral à un entier
type_to_idx = {t: i for i, t in enumerate(unique_types)}
idx = np.array([type_to_idx[t] for t in SpT])

# Crée la colormap
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=-0.5, vmax=N_types - 0.5)

# Colorbar horizontale
fig, ax = plt.subplots(figsize=(14, 0.4), layout="constrained")
cbar = fig.colorbar(
    mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
    cax=ax,
    orientation="horizontal",
    ticks=range(N_types),
)
cbar.ax.set_xticklabels(unique_types, rotation=45, ha="right")
cbar.set_label("Spectral Type", fontsize=12,labelpad=10)

plt.show()

In [ ]:
print(unique_types)
print(len(unique_types))

In [ ]:
print(type_to_idx)

In [ ]:
print(idx) 

## Overall stat

In [ ]:
df_targets_mag.head() 

In [ ]:
sorted_targets =  list(df_targets_mag.index) 
print(sorted_targets)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(16,10))
ax1,ax2,ax3,ax4 = axs.flatten()
    
df_spec_sel.hist("PWV [mm]_err_x",ax=ax1,bins=50,range=(0.,0.1),facecolor="b")
df_spec_sel.hist("PWV [mm]_err_y",ax=ax2,bins=50,range=(0.,0.1),facecolor="b")

colors = {'empty': "blue", 'FELH0600': "purple",'OG550_65mm_1':"red"}
           
for filt, group in df_spec_sel.groupby("FILTER"):
    group.hist("PWV [mm]_err_x",ax=ax3,bins=50,range=(0.,.1),facecolor=colors[filt],label=filt,alpha=0.5)
    group.hist("PWV [mm]_err_y",ax=ax4,bins=50,range=(0.,.1),facecolor=colors[filt],label=filt,alpha=0.5)
    
ax3.legend() 
ax4.legend()   
            
plt.suptitle(f" Statistical error on PWV Error {tag}")
plt.tight_layout()
plt.show()

### Dispersion des erreurs par filtres

In [ ]:
df_spec_sel_err = df_spec_sel[["PWV [mm]_err_x","PWV [mm]_err_y","TARGET","FILTER","AIRMASS","CHI2_FIT"]]

In [ ]:
df_spec_sel_err["TARGET"].unique() 

In [ ]:
# order pf selected target by magnitude Y
order_selected_targets = [t for t in sorted_targets if t in df_spec_sel_err["TARGET"].unique()]
print(order_selected_targets)

In [ ]:
plt.figure(figsize=(20, 6))
sns.boxplot(data=df_spec_sel_err, x='TARGET', y='PWV [mm]_err_x', palette='tab20')
sns.stripplot(data=df_spec_sel_err, x='TARGET', y='PWV [mm]_err_x', color='black', alpha=0.5)

sns.boxplot(
    data=df_spec_sel_err,
    x='TARGET',
    y='PWV [mm]_err_x',
    palette='tab20',
    order=order_selected_targets   # 👈 on force l’ordre ici
)
sns.stripplot(
    data=df_spec_sel_err,
    x='TARGET',
    y='PWV [mm]_err_x',
    color='black',
    alpha=0.5,
    order= order_selected_targets   # 👈 à faire aussi pour que les points suivent le même ordre
)


plt.ylabel('PWV error [mm]')
plt.title('PWV errors per Target')
plt.ylim(0.,.1)
plt.xticks(rotation=90)  # ⬅️ rotation des labels de l’axe X
plt.show()

In [ ]:
df_spec_sel_err_og550 = df_spec_sel_err[df_spec_sel_err["FILTER"] == "OG550_65mm_1"]
df_spec_sel_err_empty = df_spec_sel_err[df_spec_sel_err["FILTER"] == "empty"]

In [ ]:
plt.figure(figsize=(20, 6))

sns.boxplot(
    data=df_spec_sel_err_og550,
    x='TARGET',
    y='PWV [mm]_err_x',
    palette='tab20',
    order= order_selected_targets   # 👈 on force l’ordre ici
)
sns.stripplot(
    data=df_spec_sel_err_og550,
    x='TARGET',
    y='PWV [mm]_err_x',
    color='black',
    alpha=0.5,
    order=order_selected_targets   # 👈 à faire aussi pour que les points suivent le même ordre
)

plt.ylabel('PWV error [mm]')
plt.title('PWV errors per Target for filter OG550')
plt.ylim(0.,.05)
plt.xticks(rotation=90)  # ⬅️ rotation des labels de l’axe X
plt.show()

In [ ]:
plt.figure(figsize=(20, 6))

sns.boxplot(
    data=df_spec_sel_err_empty,
    x='TARGET',
    y='PWV [mm]_err_x',
    palette='tab20',
    order= order_selected_targets   # 👈 on force l’ordre ici
)
sns.stripplot(
    data=df_spec_sel_err_empty,
    x='TARGET',
    y='PWV [mm]_err_x',
    color='black',
    alpha=0.5,
    order=order_selected_targets   # 👈 à faire aussi pour que les points suivent le même ordre
)

plt.ylabel('PWV error [mm]')
plt.title('PWV errors per Target for filter empty')
plt.ylim(0.,.05)
plt.xticks(rotation=90)  # ⬅️ rotation des labels de l’axe X
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(20,12),layout="constrained")
ax1,ax2= axs.flatten()

sns.boxplot(data=df_spec_sel_err, x='TARGET', y='PWV [mm]_err_x', palette='tab20',ax=ax1,order= order_selected_targets )
sns.stripplot(data=df_spec_sel_err, x='TARGET', y='PWV [mm]_err_x', color='black', alpha=0.5,ax=ax1,order= order_selected_targets )
ax1.set_ylabel('PWV error [mm]')
ax1.set_title('PWV errors per Target in Spectrogram')
ax1.set_ylim(0.,.1)
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=90)  # ✅ rotation ici


sns.boxplot(data=df_spec_sel_err, x='TARGET', y='PWV [mm]_err_y', palette='tab20',ax=ax2,order= order_selected_targets )
sns.stripplot(data=df_spec_sel_err, x='TARGET', y='PWV [mm]_err_y', color='black', alpha=0.5,ax=ax2,order= order_selected_targets )
ax2.set_ylabel('PWV error [mm]')
ax2.set_title('PWV errors per Target in Spectrum')
ax2.set_ylim(0.,.2)
ax2.set_xticklabels(ax1.get_xticklabels(), rotation=90)  # ✅ rotation ici

plt.show()

### Impact de la magnitude en Y sur PWV

In [ ]:
# Palette de 20 couleurs (tu peux aussi augmenter le nombre si tu as >20 cibles)
#palette = sns.color_palette("tab20", n_colors=len(order_selected_targets))

# Créer un dictionnaire {target: couleur}
#color_map = dict(zip(order_selected_targets, palette))

In [ ]:
# correspondance type spectral → indice couleur
type_to_color_idx = {t: i for i, t in enumerate(unique_types)}
color_indices = np.array([type_to_color_idx[t] for t in SpT])

# colormap cohérente avec le diagramme couleur-couleur
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=-0.5, vmax=N_types - 0.5)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12, 6),layout="constrained")

for index,targetname in enumerate(order_selected_targets):
    the_spectype = df_targets_mag.loc[targetname,"Sp_T"]
    #the_col = color_map[targetname]
    the_col = cmap(norm(type_to_color_idx[the_spectype]))
    
    the_df = df_spec_sel_err_og550[ df_spec_sel_err_og550['TARGET'] == targetname]
    Yvalues =  the_df['PWV [mm]_err_x'].values
    Xvalue = df_targets_mag.loc[targetname]["y"]
        # répéter la valeur X autant de fois qu'il y a de Y
    Xvalues = [Xvalue] * len(Yvalues)
    
    ax.scatter(Xvalues, Yvalues, color=the_col, alpha=0.5, label=targetname,marker='o',s=100)

    # Calcul de la médiane
    Ymed = np.median(Yvalues)
    
    # Scatter de la médiane avec point plus gros
    ax.scatter(Xvalue, Ymed, color=the_col, s=200, edgecolor='black', zorder=5, label=targetname)

ax.set_xlabel("Magnitude y (mag)")
ax.set_ylabel("PWV error [mm]")
ax.set_title("PWV error vs y magnitude (par target) in filter OG550")

handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(0.01, 1.0), ncols=4,loc='upper left',fontsize=12)

# --- Colorbar cohérente ---
sm = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])  # nécessaire pour compatibilité

cbar = plt.colorbar(sm, ax=ax, ticks=range(N_types))
cbar.ax.set_yticklabels(unique_types)
cbar.set_label("Spectral Type", fontsize=12)
cbar.ax.invert_yaxis()  # O bleu en haut, M rouge en bas

ax.set_ylim(0.,0.06)
ax.grid(True, alpha=0.3)
#plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1, figsize=(12, 6), layout="constrained")

for idx, targetname in enumerate(order_selected_targets):
    #the_col = color_map[targetname]
    the_spectype = df_targets_mag.loc[targetname,"Sp_T"]
    the_col = cmap(norm(type_to_color_idx[the_spectype]))
    
    the_df = df_spec_sel_err_og550[df_spec_sel_err_og550['TARGET'] == targetname]
    Yvalues = the_df['PWV [mm]_err_x'].values
    Xvalue = df_targets_mag.loc[targetname]["y"]
    
    # Ajouter un petit jitter horizontal pour visualiser les points
    jitter = np.random.normal(0, 0.02, size=len(Yvalues))  # +/- 0.01 mag
    Xvalues = Xvalue + jitter
    
    # Scatter des points individuels
    ax.scatter(Xvalues, Yvalues, color=the_col, alpha=0.5, s=80, marker='o')
    
    # Médiane avec ligne horizontale
    Ymed = np.median(Yvalues)
    ax.hlines(Ymed, Xvalue - 0.1, Xvalue + 0.1, colors=the_col, linewidth=5, zorder=5)

# Axes et labels
ax.set_xlabel("Magnitude y (mag)")
ax.set_ylabel("PWV error [mm]")
ax.set_title("PWV error vs y magnitude (par target) in filter OG550")
ax.set_ylim(0., 0.06)
ax.grid(True, alpha=0.3)

# Légende avec une entrée par target
for targetname in order_selected_targets:
    #ax.scatter([], [], color=color_map[targetname], label=targetname)
    ax.scatter([], [], color=cmap(norm(type_to_color_idx[df_targets_mag.loc[targetname,"Sp_T"]])), label=targetname)
ax.legend(bbox_to_anchor=(0.01, 1.0), loc='upper left', ncols=4,title="Targets",fontsize=14)

# --- Colorbar cohérente ---
sm = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])  # nécessaire pour compatibilité

cbar = plt.colorbar(sm, ax=ax, ticks=range(N_types))
cbar.ax.set_yticklabels(unique_types)
cbar.set_label("Spectral Type", fontsize=12)
cbar.ax.invert_yaxis()  # O bleu en haut, M rouge en bas

plt.show()


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 6), layout="constrained")

for targetname in order_selected_targets:
    #the_col = color_map[targetname]
    the_spectype = df_targets_mag.loc[targetname,"Sp_T"]
    the_col = cmap(norm(type_to_color_idx[the_spectype]))
    
    # Sélection des données pour la target
    the_df = df_spec_sel_err_og550[df_spec_sel_err_og550['TARGET'] == targetname]
    Yvalues = the_df['PWV [mm]_err_x'].values
    Xvalue = df_targets_mag.loc[targetname]["y"]
    
    # Ajouter un petit jitter horizontal pour mieux voir les points
    jitter = np.random.normal(0, 0.012, size=len(Yvalues))  # +/- 0.012 mag
    Xvalues = Xvalue + jitter
    
    # Scatter des points individuels
    ax.scatter(Xvalues, Yvalues, color=the_col, alpha=0.5, s=80, marker='o')
    
    # Médiane
    Ymed = np.median(Yvalues)
    ax.hlines(Ymed, Xvalue - 0.1, Xvalue + 0.1, colors=the_col, linewidth=3, zorder=5)
    
    # Annoter la médiane au-dessus de la ligne
    ax.text(Xvalue, Ymed + 0.003, f"{Ymed:.3f}", color=the_col,
            ha='center', va='bottom', fontsize=16)

# Axes et labels
ax.set_xlabel("Magnitude y (mag)", fontsize=12)
ax.set_ylabel("PWV error [mm]", fontsize=12)
ax.set_title("PWV error vs y magnitude (par target) in filter OG550", fontsize=14)
ax.set_ylim(0., 0.06)
ax.grid(True, alpha=0.3, linestyle='--')

# Légende propre avec une entrée par target
for targetname in order_selected_targets:
    #ax.scatter([], [], color=color_map[targetname], label=targetname)
    ax.scatter([], [], color=cmap(norm(type_to_color_idx[df_targets_mag.loc[targetname,"Sp_T"]])), label=targetname)
ax.legend(bbox_to_anchor=(0.01, 1.0), loc='upper left', title="Targets", ncols=4,fontsize=14)

# --- Colorbar cohérente ---
sm = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])  # nécessaire pour compatibilité

cbar = plt.colorbar(sm, ax=ax, ticks=range(N_types))
cbar.ax.set_yticklabels(unique_types)
cbar.set_label("Spectral Type", fontsize=12)
cbar.ax.invert_yaxis()  # O bleu en haut, M rouge en bas


plt.show()


In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12, 6),layout="constrained")

for index,targetname in enumerate(order_selected_targets):
    #the_col = color_map[targetname]
    the_spectype = df_targets_mag.loc[targetname,"Sp_T"]
    the_col = cmap(norm(type_to_color_idx[the_spectype]))
    
    the_df = df_spec_sel_err_empty[ df_spec_sel_err_empty['TARGET'] == targetname]
    Yvalues =  the_df['PWV [mm]_err_x'].values
    Xvalue = df_targets_mag.loc[targetname]["y"]
        # répéter la valeur X autant de fois qu'il y a de Y
    Xvalues = [Xvalue] * len(Yvalues)
    
    ax.scatter(Xvalues, Yvalues, color=the_col, alpha=0.5, label=targetname,marker='o',s=100)

    # Calcul de la médiane
    Ymed = np.median(Yvalues)
    
    # Scatter de la médiane avec point plus gros
    ax.scatter(Xvalue, Ymed, color=the_col, s=200, edgecolor='black', label=targetname,zorder=5)

ax.set_xlabel("Magnitude y (mag)")
ax.set_ylabel("PWV error [mm]")
ax.set_title("PWV error vs y magnitude (par target) in filter empty")

handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(0.01, 1.0), ncols=4,loc='upper left',fontsize=14)

# --- Colorbar cohérente ---
sm = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])  # nécessaire pour compatibilité

cbar = plt.colorbar(sm, ax=ax, ticks=range(N_types))
cbar.ax.set_yticklabels(unique_types)
cbar.set_label("Spectral Type", fontsize=12)
cbar.ax.invert_yaxis()  # O bleu en haut, M rouge en bas

ax.set_ylim(0.,0.1)
ax.grid(True, alpha=0.3)
#plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1, figsize=(12, 6), layout="constrained")

for index, targetname in enumerate(order_selected_targets):
    #the_col = color_map[targetname]
    the_spectype = df_targets_mag.loc[targetname,"Sp_T"]
    the_col = cmap(norm(type_to_color_idx[the_spectype]))
    
    the_df = df_spec_sel_err_empty[df_spec_sel_err_empty['TARGET'] == targetname]
    Yvalues = the_df['PWV [mm]_err_x'].values
    Xvalue = df_targets_mag.loc[targetname]["y"]
    
    # Ajouter un petit jitter horizontal pour visualiser les points
    jitter = np.random.normal(0, 0.02, size=len(Yvalues))  # +/- 0.01 mag
    Xvalues = Xvalue + jitter
    
    # Scatter des points individuels
    ax.scatter(Xvalues, Yvalues, color=the_col, alpha=0.5, s=80, marker='o')
    
    # Médiane avec ligne horizontale
    Ymed = np.median(Yvalues)
    ax.hlines(Ymed, Xvalue - 0.1, Xvalue + 0.1, colors=the_col, linewidth=5, zorder=5)

# Axes et labels
ax.set_xlabel("Magnitude y (mag)")
ax.set_ylabel("PWV error [mm]")
ax.set_title("PWV error vs y magnitude (par target) in filter empty")
ax.set_ylim(0., 0.1)
ax.grid(True, alpha=0.3)

# Légende avec une entrée par target
for targetname in order_selected_targets:
    #ax.scatter([], [], color=color_map[targetname], label=targetname)
    ax.scatter([], [], color=cmap(norm(type_to_color_idx[df_targets_mag.loc[targetname,"Sp_T"]])), label=targetname)
ax.legend(bbox_to_anchor=(0.01, 1.0), loc='upper left', title="Targets",ncols=4,fontsize=14)

# --- Colorbar cohérente ---
sm = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])  # nécessaire pour compatibilité

cbar = plt.colorbar(sm, ax=ax, ticks=range(N_types))
cbar.ax.set_yticklabels(unique_types)
cbar.set_label("Spectral Type", fontsize=12)
cbar.ax.invert_yaxis()  # O bleu en haut, M rouge en bas

plt.show()


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 6), layout="constrained")

for targetname in order_selected_targets:
    #the_col = color_map[targetname]
    the_spectype = df_targets_mag.loc[targetname,"Sp_T"]
    the_col = cmap(norm(type_to_color_idx[the_spectype]))
    
    # Sélection des données pour la target
    the_df = df_spec_sel_err_empty[df_spec_sel_err_empty['TARGET'] == targetname]
    Yvalues = the_df['PWV [mm]_err_x'].values
    Xvalue = df_targets_mag.loc[targetname]["y"]
    
    # Ajouter un petit jitter horizontal pour mieux voir les points
    jitter = np.random.normal(0, 0.012, size=len(Yvalues))  # +/- 0.012 mag
    Xvalues = Xvalue + jitter
    
    # Scatter des points individuels
    ax.scatter(Xvalues, Yvalues, color=the_col, alpha=0.5, s=80, marker='o')
    
    # Médiane
    Ymed = np.median(Yvalues)
    ax.hlines(Ymed, Xvalue - 0.1, Xvalue + 0.1, colors=the_col, linewidth=3, zorder=5)
    
    # Annoter la médiane au-dessus de la ligne
    ax.text(Xvalue, Ymed + 0.003, f"{Ymed:.3f}", color=the_col,
            ha='center', va='bottom', fontsize=16)

# Axes et labels
ax.set_xlabel("Magnitude y (mag)", fontsize=12)
ax.set_ylabel("PWV error [mm]", fontsize=12)
ax.set_title("PWV error vs y magnitude (par target) in filter empty", fontsize=14)
ax.set_ylim(0., 0.10)
ax.grid(True, alpha=0.3, linestyle='--')

# Légende propre avec une entrée par target
for targetname in order_selected_targets:
    #ax.scatter([], [], color=color_map[targetname], label=targetname)
    ax.scatter([], [], color=cmap(norm(type_to_color_idx[df_targets_mag.loc[targetname,"Sp_T"]])), label=targetname)
ax.legend(bbox_to_anchor=(0.01, 1.0), loc='upper left', title="Targets", ncols=5,fontsize=14)

# --- Colorbar cohérente ---
sm = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])  # nécessaire pour compatibilité

cbar = plt.colorbar(sm, ax=ax, ticks=range(N_types))
cbar.ax.set_yticklabels(unique_types)
cbar.set_label("Spectral Type", fontsize=12)
cbar.ax.invert_yaxis()  # O bleu en haut, M rouge en bas

plt.show()


## Relation wrt Colors

In [ ]:
SedColors = ["u-g","g-r","r-i","i-z","z-y"]

NSedColors = len(SedColors)

fig, axs = plt.subplots(NSedColors, 2, figsize=(18, NSedColors*4), layout="constrained")

axs_flt = axs.flatten()

for idxsedcol,sedcolor in enumerate(SedColors): 

    ## with filter OG550
    ax = axs_flt[idxsedcol*2]

    for targetname in order_selected_targets:
        #the_col = color_map[targetname]
        the_spectype = df_targets_mag.loc[targetname,"Sp_T"]
        the_col = cmap(norm(type_to_color_idx[the_spectype]))
    
        # Sélection des données pour la target
        the_df = df_spec_sel_err_og550[df_spec_sel_err_og550['TARGET'] == targetname]
        Yvalues = the_df['PWV [mm]_err_x'].values
        Xvalue = df_targets_mag.loc[targetname][sedcolor]
    
        # Ajouter un petit jitter horizontal pour mieux voir les points
        jitter = np.random.normal(0, 0.003, size=len(Yvalues))  # +/- 0.012 mag
        Xvalues = Xvalue + jitter
    
        # Scatter des points individuels
        ax.scatter(Xvalues, Yvalues, color=the_col, alpha=0.5, s=80, marker='o')
    
        # Médiane
        Ymed = np.median(Yvalues)
        ax.hlines(Ymed, Xvalue - 0.01, Xvalue + 0.01, colors=the_col, linewidth=3, zorder=5)
    
        # Annoter la médiane au-dessus de la ligne
        ax.text(Xvalue, Ymed + 0.003, f"{Ymed:.3f}", color=the_col,ha='center', va='bottom', fontsize=16)

    # Axes et labels
    ax.set_xlabel(f"{sedcolor} (mag)", fontsize=12)
    ax.set_ylabel("PWV error [mm]", fontsize=12)
    ax.set_title(f"PWV error vs {sedcolor} (per target) in filter OG550", fontsize=14)
    ax.set_ylim(0., 0.08)
    ax.grid(True, alpha=0.3, linestyle='--')

    if idxsedcol == NSedColors-1:
        
    # Légende propre avec une entrée par target
        for targetname in order_selected_targets:
            #ax.scatter([], [], color=color_map[targetname], label=targetname)
            ax.scatter([], [], color=cmap(norm(type_to_color_idx[df_targets_mag.loc[targetname,"Sp_T"]])), label=targetname)
        #ax.legend(bbox_to_anchor=(1.01, 1.0), loc='upper left', title="Targets", fontsize=10)
        ax.legend( loc='upper left', title="Targets",ncols=5, fontsize=10)

    ## with filter emtpy
    ax = axs_flt[idxsedcol*2+1]
    
    for targetname in order_selected_targets:
        #the_col = color_map[targetname]
        the_spectype = df_targets_mag.loc[targetname,"Sp_T"]
        the_col = cmap(norm(type_to_color_idx[the_spectype]))
    
        # Sélection des données pour la target
        the_df = df_spec_sel_err_empty[df_spec_sel_err_empty['TARGET'] == targetname]
        Yvalues = the_df['PWV [mm]_err_x'].values
        Xvalue = df_targets_mag.loc[targetname][sedcolor]
    
        # Ajouter un petit jitter horizontal pour mieux voir les points
        jitter = np.random.normal(0, 0.003, size=len(Yvalues))  # +/- 0.012 mag
        Xvalues = Xvalue + jitter
    
        # Scatter des points individuels
        ax.scatter(Xvalues, Yvalues, color=the_col, alpha=0.5, s=80, marker='o')
    
        # Médiane
        Ymed = np.median(Yvalues)
        ax.hlines(Ymed, Xvalue - 0.01, Xvalue + 0.01, colors=the_col, linewidth=3, zorder=5)
    
        # Annoter la médiane au-dessus de la ligne
        ax.text(Xvalue, Ymed + 0.003, f"{Ymed:.3f}", color=the_col,ha='center', va='bottom', fontsize=16)

    # Axes et labels
    ax.set_xlabel(f"{sedcolor} (mag)", fontsize=12)
    ax.set_ylabel("PWV error [mm]", fontsize=12)
    ax.set_title(f"PWV error vs {sedcolor} (per target) in filter empty", fontsize=14)
    ax.set_ylim(0., 0.08)
    ax.grid(True, alpha=0.3, linestyle='--') 

    if idxsedcol == NSedColors-1:
        
    # Légende propre avec une entrée par target
        for targetname in order_selected_targets:
            #ax.scatter([], [], color=color_map[targetname], label=targetname)
            ax.scatter([], [], color=cmap(norm(type_to_color_idx[df_targets_mag.loc[targetname,"Sp_T"]])), label=targetname)
        #ax.legend(bbox_to_anchor=(1.01, 1.0), loc='upper left', title="Targets", fontsize=10)
        ax.legend( loc='upper left', title="Targets",ncols=5, fontsize=10)



# --- Colorbar cohérente sur toute la figure ---
#sm = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
#sm.set_array([])

# La colorbar s'applique à tous les subplots
#cbar = fig.colorbar(sm, ax=axs.ravel().tolist(), ticks=range(N_types), fraction=0.02, pad=0.02)
#cbar.ax.set_yticklabels(unique_types)
#cbar.set_label("Spectral Type", fontsize=12)
#cbar.ax.invert_yaxis()


# --- Colorbar sur toute la hauteur, à gauche ---
sm = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])

# Position (x0, y0, largeur, hauteur) en coordonnées figure [0–1]
cbar_ax = fig.add_axes([1.01, 0.1, 0.015, 0.8])  # ajuster à ton goût
cbar = fig.colorbar(sm, cax=cbar_ax, ticks=range(N_types))
cbar.ax.set_yticklabels(unique_types)
cbar.set_label("Spectral Type", fontsize=12)
cbar.ax.invert_yaxis()

plt.show()


### Relation with Chi2

In [ ]:
# Palette de 20 couleurs (tu peux aussi augmenter le nombre si tu as >20 cibles)
#palette = sns.color_palette("tab20", n_colors=len(order_selected_targets))

# Créer un dictionnaire {target: couleur}
#color_map = dict(zip(order_selected_targets, palette))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6), layout="constrained")

for targetname in order_selected_targets:
    #the_col = color_map[targetname]
    the_spectype = df_targets_mag.loc[targetname,"Sp_T"]
    the_col = cmap(norm(type_to_color_idx[the_spectype]))
    
    # Sélection des données pour la target
    the_df = df_spec_sel_err_og550[df_spec_sel_err_og550['TARGET'] == targetname]
    Yvalues = the_df['PWV [mm]_err_x'].values
    Xvalues = the_df['CHI2_FIT'].values
       
    # Scatter des points individuels
    ax.scatter(Xvalues, Yvalues, color=the_col, alpha=0.5, s=120, marker='o')
    
    # Médians
    Ymed = np.median(Yvalues)
    Xmed = np.median(Xvalues)
    ax.scatter(Xmed, Ymed, color=the_col, s=200, edgecolor='black', zorder=5, label=targetname)
    
    # Annoter la médiane au-dessus de la ligne
    #ax.text(Xvalue, Ymed + 0.003, f"{Ymed:.3f}", color=the_col,
    #        ha='center', va='bottom', fontsize=16)

# Axes et labels
ax.set_xlabel("CHI2_FIT", fontsize=12)
ax.set_ylabel("PWV error [mm]", fontsize=12)
ax.set_title("PWV error vs CHI2_FIT (par target) in filter OG550", fontsize=14)
ax.set_ylim(0., 0.06)
ax.grid(True, alpha=0.3, linestyle='--')

# Légende propre avec une entrée par target
for targetname in order_selected_targets:
    #ax.scatter([], [], color=color_map[targetname], label=targetname)
    ax.scatter([], [], color=cmap(norm(type_to_color_idx[df_targets_mag.loc[targetname,"Sp_T"]])), label=targetname)

handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(0.01, 1.0), ncols=5,loc='upper left',fontsize=12)

# --- Colorbar cohérente ---
sm = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])  # nécessaire pour compatibilité

cbar = plt.colorbar(sm, ax=ax, ticks=range(N_types))
cbar.ax.set_yticklabels(unique_types)
cbar.set_label("Spectral Type", fontsize=12)
cbar.ax.invert_yaxis()  # O bleu en haut, M rouge en bas

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6), layout="constrained")

for targetname in order_selected_targets:
    #the_col = color_map[targetname]
    the_spectype = df_targets_mag.loc[targetname,"Sp_T"]
    the_col = cmap(norm(type_to_color_idx[the_spectype]))
    
    # Sélection des données pour la target
    the_df = df_spec_sel_err_empty[df_spec_sel_err_empty['TARGET'] == targetname]
    Yvalues = the_df['PWV [mm]_err_x'].values
    Xvalues = the_df['CHI2_FIT'].values
    
    
    # Scatter des points individuels
    ax.scatter(Xvalues, Yvalues, color=the_col, alpha=0.5, s=120, marker='o')
    
    # Médians
    Ymed = np.median(Yvalues)
    Xmed = np.median(Xvalues)
    ax.scatter(Xmed, Ymed, color=the_col, s=200, edgecolor='black', zorder=5, label=targetname)
    
    # Annoter la médiane au-dessus de la ligne
    #ax.text(Xvalue, Ymed + 0.003, f"{Ymed:.3f}", color=the_col,
    #        ha='center', va='bottom', fontsize=16)

# Axes et labels
ax.set_xlabel("CHI2_FIT", fontsize=12)
ax.set_ylabel("PWV error [mm]", fontsize=12)
ax.set_title("PWV error vs CHI2_FIT (par target) in filter empty", fontsize=14)
ax.set_ylim(0., 0.1)
ax.grid(True, alpha=0.3, linestyle='--')

# Légende propre avec une entrée par target
for targetname in order_selected_targets:
    #ax.scatter([], [], color=color_map[targetname], label=targetname)
    ax.scatter([], [], color=cmap(norm(type_to_color_idx[df_targets_mag.loc[targetname,"Sp_T"]])), label=targetname)

handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(0.01, 1.0), ncols=5,loc='upper left',fontsize=12)

# --- Colorbar cohérente ---
sm = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])  # nécessaire pour compatibilité

cbar = plt.colorbar(sm, ax=ax, ticks=range(N_types))
cbar.ax.set_yticklabels(unique_types)
cbar.set_label("Spectral Type", fontsize=12)
cbar.ax.invert_yaxis()  # O bleu en haut, M rouge en bas

plt.show()

### Impact de l'airmass sur les erreurs en PWV

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 8), layout="constrained")

for targetname in order_selected_targets:
    #the_col = color_map[targetname]
    the_spectype = df_targets_mag.loc[targetname,"Sp_T"]
    the_col = cmap(norm(type_to_color_idx[the_spectype]))
    
    # Sélection des données pour la target
    the_df = df_spec_sel_err_og550[df_spec_sel_err_og550['TARGET'] == targetname]
    Yvalues = the_df['PWV [mm]_err_x'].values
    Xvalues = the_df['AIRMASS'].values
       
    # Scatter des points individuels
    ax.scatter(Xvalues, Yvalues, color=the_col, alpha=0.5, s=120, marker='o')
    
    # Médians
    Ymed = np.median(Yvalues)
    Xmed = np.median(Xvalues)
    ax.scatter(Xmed, Ymed, color=the_col, s=200, edgecolor='black', zorder=5, label=targetname)
    
    # Annoter la médiane au-dessus de la ligne
    #ax.text(Xvalue, Ymed + 0.003, f"{Ymed:.3f}", color=the_col,
    #        ha='center', va='bottom', fontsize=16)

# Axes et labels
ax.set_xlabel("airmass", fontsize=12)
ax.set_ylabel("PWV error [mm]", fontsize=12)
ax.set_title("PWV error vs airmass (par target) in filter OG550", fontsize=14)
ax.set_ylim(0., 0.06)
ax.grid(True, alpha=0.3, linestyle='--')

# Légende propre avec une entrée par target
for targetname in order_selected_targets:
    #ax.scatter([], [], color=color_map[targetname], label=targetname)
    ax.scatter([], [], color=cmap(norm(type_to_color_idx[df_targets_mag.loc[targetname,"Sp_T"]])), label=targetname)

handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(0.01, 1.0), ncols=5,loc='upper left',fontsize=14)

# --- Colorbar cohérente ---
sm = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])  # nécessaire pour compatibilité

cbar = plt.colorbar(sm, ax=ax, ticks=range(N_types))
cbar.ax.set_yticklabels(unique_types)
cbar.set_label("Spectral Type", fontsize=12)
cbar.ax.invert_yaxis()  # O bleu en haut, M rouge en bas

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 8), layout="constrained")

for targetname in order_selected_targets:
    #the_col = color_map[targetname]
    the_spectype = df_targets_mag.loc[targetname,"Sp_T"]
    the_col = cmap(norm(type_to_color_idx[the_spectype]))
    
    
    # Sélection des données pour la target
    the_df = df_spec_sel_err_empty[df_spec_sel_err_empty['TARGET'] == targetname]
    Yvalues = the_df['PWV [mm]_err_x'].values
    Xvalues = the_df['AIRMASS'].values
       
    # Scatter des points individuels
    ax.scatter(Xvalues, Yvalues, color=the_col, alpha=0.5, s=120, marker='o')
    
    # Médians
    Ymed = np.median(Yvalues)
    Xmed = np.median(Xvalues)
    ax.scatter(Xmed, Ymed, color=the_col, s=200, edgecolor='black', zorder=5, label=targetname)
    
    # Annoter la médiane au-dessus de la ligne
    #ax.text(Xvalue, Ymed + 0.003, f"{Ymed:.3f}", color=the_col,
    #        ha='center', va='bottom', fontsize=16)

# Axes et labels
ax.set_xlabel("airmass", fontsize=12)
ax.set_ylabel("PWV error [mm]", fontsize=12)
ax.set_title("PWV error vs airmass (par target) in filter empty", fontsize=14)
ax.set_ylim(0., 0.08)
ax.grid(True, alpha=0.3, linestyle='--')

# Légende propre avec une entrée par target
for targetname in order_selected_targets:
    #ax.scatter([], [], color=color_map[targetname], label=targetname)
    ax.scatter([], [], color=cmap(norm(type_to_color_idx[df_targets_mag.loc[targetname,"Sp_T"]])), label=targetname)

handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(0.01, 1.0), ncols=5,loc='upper left',fontsize=12)

# --- Colorbar cohérente ---
sm = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])  # nécessaire pour compatibilité

cbar = plt.colorbar(sm, ax=ax, ticks=range(N_types))
cbar.ax.set_yticklabels(unique_types)
cbar.set_label("Spectral Type", fontsize=12)
cbar.ax.invert_yaxis()  # O bleu en haut, M rouge en bas

plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.scatterplot(data=df_spec_sel_err, x='AIRMASS', y='PWV [mm]_err_x', hue='TARGET', palette='tab10')
plt.ylabel('PWV error [mm]')
plt.title('PWV stat error wrt AIRMASS, colored per TARGET')
plt.ylim(0.,.1)

# Déplacer la légende à gauche en dehors du plot
plt.legend(
    bbox_to_anchor=(1.01, 0.5),  # position : (x, y)
    loc='center left',           # ancrage de la légende
    title='TARGET',
    ncols=3
)

plt.tight_layout()

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 8), layout="constrained")

for targetname in order_selected_targets:
    #the_col = color_map[targetname]
    the_spectype = df_targets_mag.loc[targetname,"Sp_T"]
    the_col = cmap(norm(type_to_color_idx[the_spectype]))
    
    # Sélection des données pour la target
    the_df = df_spec_sel_err_og550[df_spec_sel_err_og550['TARGET'] == targetname]
    Yvalues = the_df['CHI2_FIT'].values
    Xvalues = the_df['AIRMASS'].values
       
    # Scatter des points individuels
    ax.scatter(Xvalues, Yvalues, color=the_col, alpha=0.5, s=120, marker='o')
    
    # Médians
    Ymed = np.median(Yvalues)
    Xmed = np.median(Xvalues)
    ax.scatter(Xmed, Ymed, color=the_col, s=200, edgecolor='black', zorder=5, label=targetname)
    
    # Annoter la médiane au-dessus de la ligne
    #ax.text(Xvalue, Ymed + 0.003, f"{Ymed:.3f}", color=the_col,
    #        ha='center', va='bottom', fontsize=16)

# Axes et labels
ax.set_xlabel("airmass", fontsize=12)
ax.set_ylabel("CHI2_FIT", fontsize=12)
ax.set_title("CHI2_FIT vs airmass (par target) in filter OG550", fontsize=14)
#ax.set_ylim(0., 0.06)
ax.grid(True, alpha=0.3, linestyle='--')

# Légende propre avec une entrée par target
for targetname in order_selected_targets:
    #ax.scatter([], [], color=color_map[targetname], label=targetname)
    ax.scatter([], [], color=cmap(norm(type_to_color_idx[df_targets_mag.loc[targetname,"Sp_T"]])), label=targetname)

handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(0.01, 1.0), ncols=5,loc='upper left',fontsize=12)

# --- Colorbar cohérente ---
sm = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])  # nécessaire pour compatibilité

cbar = plt.colorbar(sm, ax=ax, ticks=range(N_types))
cbar.ax.set_yticklabels(unique_types)
cbar.set_label("Spectral Type", fontsize=12)
cbar.ax.invert_yaxis()  # O bleu en haut, M rouge en bas

plt.show()

## Show time variation each night

In [ ]:
all_selected_nights = df_spec_sel["nightObs"].unique()

In [ ]:
def funclineres(params, x, y, yerr):
    # Return residual = fit-observed
    return (y-params[0] -params[1]*x)/yerr
def funcline(params,x):
    return params[0] + params[1]*x

In [ ]:
def MakeLineFit(df_night_pwv_curve):
    """
    """

    x = df_night_pwv_curve["dt"].values
    y = df_night_pwv_curve["PWV [mm]_x"].values
    yerr = df_night_pwv_curve["PWV [mm]_err_x"].values
    n = len(y)
        
    #popt, pcov = optimize.curve_fit(f, x, y, [1,-4])
    fit_res = least_squares(funclineres,[5.,0],args = (x,y,yerr))
    popt = fit_res.x 
    npar = len(popt)
    J = fit_res.jac
    cov = np.linalg.inv(J.T.dot(J))
    chi2dof = ((funclineres(popt,x,y,yerr))**2).sum()/(n-npar)
    cov *= chi2dof
    perr = np.sqrt(np.diagonal(cov)) 
    
    xfit = np.linspace(x.min()*0.99,x.max()*1.05)
    yfit = funcline(popt,xfit)
        
    slope = popt[1]
    slope_err = perr[1]

    return x,y,yerr,n,chi2dof,xfit,yfit,slope,slope_err

In [ ]:
def local_linear_interp(df):
    """
    Calcule une interpolation linéaire locale z[i] et l'erreur dz[i] = y[i]-z[i],
    en tenant compte d'un vecteur d'erreurs ey sur y.
    
    Paramètres
    ----------
    x : array-like (datetime64 ou float)
        Les abscisses.
    y : array-like
        Les ordonnées observées.
    ey : array-like ou None
        Erreurs sur y. Si None, on renvoie ez=None.
    
    Retour
    ------
    z : np.ndarray
        Valeurs interpolées.
    dz : np.ndarray
        Différences y - z.
    ez : np.ndarray or None
        Erreurs propagées sur z.
    """

    

    x = df["Time"].values
    y = df["PWV [mm]_x"].values
    ey = df["PWV [mm]_err_x"].values
   
    
    #x = np.asarray(x)
    #y = np.asarray(y)
    N = len(x)

    #if ey is not None:
    #    ey = np.asarray(ey)

    # Convertit x en secondes si c'est un datetime64
    if np.issubdtype(x.dtype, np.datetime64):
        x_sec = (x - x[0]) / np.timedelta64(1, 's')
    else:
        x_sec = x.astype(float)

    z = np.full(N, np.nan)
    ez = np.full(N, np.nan) if ey is not None else None

    for i in range(1, N - 1):
        dx = x_sec[i+1] - x_sec[i-1]
        if dx == 0:
            continue  # évite division par zéro
        alpha = (x_sec[i] - x_sec[i-1]) / dx

        # Interpolation
        z[i] = y[i-1] + (y[i+1] - y[i-1]) * alpha

        # Erreur propagée
        if ey is not None:
            ez[i] = np.sqrt((1 - alpha)**2 * ey[i-1]**2 + alpha**2 * ey[i+1]**2)

    dz = y - z
    return x[1:-1], z[1:-1], dz[1:-1], ez[1:-1]

In [ ]:
all_selected_nights

## Plot night by night

### Select only PWV curves with OG550 only

In [ ]:
df_spec_sel = df_spec_sel[df_spec_sel["FILTER"].isin(PWV_FILTEROG550_LIST) ]

### Big loop over nights

In [ ]:
all_dateObs_sel = {}
    
# loop on nights
for night in all_selected_nights:
    #select the night
    df_spec_night = df_spec_sel[df_spec_sel["nightObs"] == night]
    
    #select the variables
    df_night_pwv_curve = df_spec_night[["Time","PWV [mm]_x","PWV [mm]_err_x","FILTER","AIRMASS","ex_azimuth"]]

    tmin = df_night_pwv_curve["Time"].min()
    tmax = df_night_pwv_curve["Time"].max()

    # convert dt in hours
    df_night_pwv_curve["dt"] = (df_night_pwv_curve["Time"] - tmin).dt.total_seconds()/3600.
    
    # extract statistics on pwv
    stat = df_night_pwv_curve[["PWV [mm]_x"]].describe()
    date_form = DateFormatter("%y-%m-%dT%H:%M")
    count = int(stat.loc["count"].values[0])
    mean = stat.loc["mean"].values[0]
    median = stat.loc["50%"].values[0]
    std = stat.loc["std"].values[0]
    textstr = "\n".join((f"count : {count}",
                     f"mean : {mean:.1f} mm",
                     f"median : {median:.1f} mm",
                     f"std : {std:.1f} mm",
                    ))

   
    # mid-night
    nightstr = datetime.strptime(str(night), "%Y%m%d")
    dt_midnight = get_astronomical_midnight(site_lsst, nightstr.date())
    dt_midnight = dt_midnight.replace(tzinfo=timezone.utc)
    

    ## position of midnight in the plot
    x_midnight = mdates.date2num(dt_midnight)

    #print(f" ===== NIGHT = {night} :: dt_midnight =  {dt_midnight}",type(dt_midnight),"   x_midnight = ",x_midnight,type(x_midnight))

    ## to calculate relative time wrt  midnight
    df_night_pwv_curve["delta_t_hr"] = (df_night_pwv_curve["Time"] - dt_midnight) / pd.Timedelta(hours=1)
    tmin_midnight = (tmin  - dt_midnight)/pd.Timedelta(hours=1)
    tmax_midnight = (tmax  - dt_midnight)/pd.Timedelta(hours=1)
    
    
    N= len(df_night_pwv_curve)
    #N= len(df_night_pwv_curve_empty)

    if N>8:
        # does the fit
        try:
            try:
                x,y,yerr,n,chi2dof,xfit,yfit,slope,slope_err = MakeLineFit(df_night_pwv_curve)
                xout, zout, dzout, ezout = local_linear_interp(df_night_pwv_curve)
        
            except Exception as inst:
                print(type(inst))    # the exception type
                print(inst.args)     # arguments stored in .args
                print(inst)  
                
                x=-1
                y=-1
                yerr=-1
                n=-1
                chi2dof = -1
                xfit = -1
                yfit = -1
                slope = -1
                slope_err==-1
                
            
        
            textstr2 = "\n".join((f"d(PWV/dt) : ",
                     f"slope : {slope:.3f} mm/h",
                     f"slope err : {slope_err:.3f} mm/h",
                     f"chi2/ndeg : {chi2dof:.2f}",       
                    ))

            textstr3 = "\n".join((f"d(PWV/dt) : ",
                     f"slope : {slope:.3f} mm/h",
                     f"slope err : {slope_err:.3f} mm/h",  
                    ))
    

            stat.loc["slope","PWV [mm]_x"] = slope 
            stat.loc["slope_err","PWV [mm]_x"] = slope_err 
            stat.loc["chi2","PWV [mm]_x"] = chi2dof
        
            # plot
            #fig = plt.figure(figsize=(18,8))
            fig = plt.figure(figsize=(18,12))
            #gs = GridSpec(1, 2,  width_ratios=[2,1],figure=fig)
            gs = GridSpec(2, 2,  width_ratios=[2,1],figure=fig)
            #gs = GridSpec(1, 1,figure=fig)

            #ax = fig.add_subplot(gs[0])
            ax = fig.add_subplot(gs[0,0])
            ax_bis = ax.twinx()  # crée un axe y secondaire partageant le même x
            ax_bis.invert_yaxis()

            ax_ter = ax.twinx()
            
            # Offset the right spine of ax_ter.  The ticks and label have already been
            # placed on the right by twinx above.
            ax_ter.spines.right.set_position(("axes", 1.1))
            
            #ax2 = fig.add_subplot(gs[1],sharey=ax)
            #ax2 = fig.add_subplot(gs[1])
            ax2 = fig.add_subplot(gs[0,1])
        
            leg=ax.get_legend()
            #leg2=ax2.get_legend()
      
        
            # left figure
            
            df = df_night_pwv_curve  
            
            list_all_filts = df["FILTER"].unique()
            list_all_filts= sorted(list_all_filts )

            
            # Couleurs distinctes pour chaque filtre
            colors = {filt: col for filt, col in zip(list_all_filts, plt.cm.tab10.colors)}
            colors = {filt: col for filt, col in zip(list_all_filts, ["r","b"])}
            colors = {'empty': "blue", 'FELH0600': "purple",'OG550_65mm_1':"red"}

            
            for filt, group in df.groupby("FILTER"):
                p1 = ax.errorbar(
                    group["Time"],
                    group["PWV [mm]_x"],
                    yerr=group["PWV [mm]_err_x"],
                    fmt="o",
                    label=filt,
                    color=colors[filt],
                    ecolor="k",
                    capsize=5,
                    markersize=10
                    )
            

                
                # No airmass : add confusion
                #ax_bis.scatter(group["Time"],group["AIRMASS"], color=colors[filt],marker="x",label="airmass")
                p2 = ax_bis.scatter(group["Time"],group["AIRMASS"], color="purple",marker="x",label="airmass")
                p3 = ax_ter.scatter(group["Time"],group["ex_azimuth"], color="darkblue",marker="+",label="azimuth")
                
            ax.errorbar(xout,zout,yerr=ezout,fmt="+", label="interpolated",color="darkgreen",capsize=5,markersize=5) 

            

            xx =   df["Time"].values
            yy = df["PWV [mm]_x"].values
            nn = len(xx)

            for i in range(1,nn-2):
                ax.plot( [xx[i-1], xx[i+1]], [yy[i-1],yy[i+1]], ":",color="g")
                  
            
            ax_bis.set_ylim(2.5,0.5)
            ax_bis.set_ylabel("airmass")
            ax_ter.set_ylabel("azimuth")

            ax_bis.legend(loc="lower right", handles=[p2, p3])

           

            # Couleur du label de chaque axe
            ax.yaxis.label.set_color(p1[0].get_color())   # p1 est un tuple (line, caplines, barlinecols)
            ax_bis.yaxis.label.set_color(p2.get_facecolor()[0])  # scatter -> facecolor
            ax_ter.yaxis.label.set_color(p3.get_facecolor()[0])

            # Couleur des ticks (graduations) sur chaque axe
            ax.tick_params(axis='y', colors=p1[0].get_color())
            ax_bis.tick_params(axis='y', colors=p2.get_facecolor()[0])
            ax_ter.tick_params(axis='y', colors=p3.get_facecolor()[0])

            # Couleur de la spine (la ligne verticale de l’axe)
            ax.spines['left'].set_color(p1[0].get_color())
            ax_bis.spines['right'].set_color(p2.get_facecolor()[0])
            ax_ter.spines['right'].set_color(p3.get_facecolor()[0])


            
            ax.axvline(x=x_midnight, color='k', linestyle=':', linewidth=2)
            ax.legend(loc="upper right")
            ax.grid()
            ax.set_ylabel("PWV [mm]")
            ax.set_xlabel("time")
            ax.xaxis.set_major_formatter(date_form)
            ax.xaxis_date()
            ax.set_ylim(PWVMIN,PWVMAX)
            ax.text(0.03, 0.95, textstr, transform=ax.transAxes, fontsize=14,
            verticalalignment='top', bbox=props)
            ax.text(0.35, 0.95, textstr3, transform=ax.transAxes, fontsize=14,
            verticalalignment='top', bbox=props)
            #ax.set_title(f"night {night}")
            ax.tick_params(axis="x", labelrotation=45)

           
            # right figure
            
            ax2.hist(dzout,bins=50,range=(-2,2),facecolor="b")
            ax2.set_xlabel("$\Delta PWV$ (mm)")

            mean_dzout = np.mean(dzout)
            med_dzout = np.median(dzout)
            std_dzout = np.std(dzout)
            sigiqr_dzout = std_iqr(dzout)


            textstr4 = "\n".join((" $\Delta PWV$ : ",
                     f"median : {med_dzout:.3f} mm",
                     f"std : {std_dzout:.3f} mm", 
                     f"sig_iqr : {sigiqr_dzout:.3f} mm", 
                    ))

            ax2.text(0.01, 0.95, textstr4, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)


            ## bottom left and right figure


            ## does some computations on PWV difference and azimuth differences
            # error on time difference    
            sig_dpwv = np.sqrt(2)*sigiqr_dzout
                
            xx =   df["delta_t_hr"].values
            yy = df["PWV [mm]_x"].values
            eyy = df["PWV [mm]_err_x"].values
            az = df["ex_azimuth"].values
            nn = len(xx)
                
            dpwv = np.zeros(nn-1)
            dpwv2 = np.zeros(nn-1)
            edpwv = np.zeros(nn-1)
            daz = np.zeros(nn-1)
            dtt = np.zeros(nn-1)
            avxx = np.zeros(nn-1)

            # show interploated lines
            #for i in range(1,nn-2):
            #    ax3.plot( [xx[i-1], xx[i+1]], [yy[i-1],yy[i+1]], ":",color="g")

            for i in range(0,nn-1):
                #avxx[i] = (xx[i+1] + xx[i])/2.
                avxx[i] = xx[i+1] 
                dtt[i] = (xx[i+1] - xx[i])*60.
                dpwv[i] = np.abs(yy[i+1]-yy[i])
                dpwv2[i] = yy[i+1]-yy[i]
                edpwv[i] = np.sqrt(eyy[i+1]**2 + eyy[i]**2)
                daz[i] = np.abs(delta_angle_deg(az[i+1],az[i]))


            #Keep xmin,xmax from ax (gs[0,0])
            #xmin_tmin,xmax_tmax = ax.get_xlim()
            xmin_tmim_midn = (tmin - dt_midnight)/pd.Timedelta(hours=1)
            xmax_tmax_midn = (tmax - dt_midnight)/pd.Timedelta(hours=1)
            ax3.set_xlim(xmin_tmim_midn,xmax_tmax_midn)
            
            
            ax3 = fig.add_subplot(gs[1,0])
            ax3_bis = ax3.twinx()  # crée un axe y secondaire partageant le même x
            ax3_bis.invert_yaxis()

            ax3_ter = ax3.twinx()
            
            # Offset the right spine of ax_ter.  The ticks and label have already been
            # placed on the right by twinx above.
            ax3_ter.spines.right.set_position(("axes", 1.1))
            
            #ax2 = fig.add_subplot(gs[1],sharey=ax)
            #ax2 = fig.add_subplot(gs[1])
            ax4 = fig.add_subplot(gs[1,1])

            
            #for filt, group in df.groupby("FILTER"):
                #p4 = ax3.errorbar(
                #    group["delta_t_hr"],
                #    group["PWV [mm]_x"],
                #    yerr=group["PWV [mm]_err_x"],
                #    fmt="o",
                #    label=filt,
                #    color=colors[filt],
                #    ecolor="k",
                #    capsize=5,
                #    markersize=10
                #    )
            
            #p4 = ax3.errorbar(
            #        avxx,
            #        dpwv2,
            #        yerr=edpwv,
            #        fmt="o",
            #        label="OG550",
            #        color="r",
            #        ecolor="k",
            #        capsize=5,
            #        markersize=10
            #        )

            # Normalisation des couleurs
            norm1 = plt.Normalize(vmin=np.min(daz), vmax=np.max(daz))
            #norm1 = plt.Normalize(vmin=0., vmax= 180.0)
            cmap1 = plt.cm.jet
            daz_colors = cmap1(norm1(daz))
                   
            
            # Ajout des barres d'erreur en Y, avec même couleur que le point
            for i in range(len(avxx)):
                if i == 0:
                    p4 = ax3.errorbar(
                        avxx[i], dpwv2[i],
                        yerr=edpwv[i],    # ton array d'erreurs en y
                        fmt='o',          # redessine pas le point
                        color = daz_colors[i],
                        ecolor = daz_colors[i],    # couleur = même que le point
                        elinewidth=1.5,
                        capsize=5,
                        alpha=1.0,
                        markersize=10,
                        label="OG550",
                        )
                else:
                    p99 = ax3.errorbar(
                        avxx[i], dpwv2[i],
                        yerr=edpwv[i],    # ton array d'erreurs en y
                        fmt='o',          # redessine pas le point
                        color = daz_colors[i],
                        ecolor = daz_colors[i],    # couleur = même que le point
                        elinewidth=1.5,
                        capsize=5,
                        alpha=1.0,
                        markersize=10,
                        )
                    
            # Création d’un objet mappable factice pour la colorbar
            sm = plt.cm.ScalarMappable(cmap=cmap1, norm=norm1)
            sm.set_array([])  # nécessaire pour lier la colorbar à la cmap

            # setting position
            from mpl_toolkits.axes_grid1.inset_locator import inset_axes
            # Ajoute un petit axe pour la colorbar sous la figure
            cbar_ax = fig.add_axes([0.06, -0.02, 0.5, 0.02])  # [x0, y0, largeur, hauteur]

            cbar = fig.colorbar(sm, cax=cbar_ax, orientation='horizontal')

            # Ajout de la colorbar horizontale sous ax3
            cbar3 = plt.colorbar(
                sm,
                cax =  cbar_ax, # set relative position of the colorbar wrt the image
                ax=ax3,
                orientation='horizontal',
                #pad=-0.15,      # espace entre ax3 et la colorbar
                aspect=40,     # ratio largeur/hauteur
            )

            # Label de la colorbar
            cbar3.set_label("|ΔAzimuth| [deg]", fontsize=12)

            # Optionnel : ajuster les ticks
            # cbar.set_ticks([0, 45, 90, 135, 180])

            ax3.axhline(sig_dpwv,color="k",linestyle=":")
            ax3.axhline(-sig_dpwv,color="k",linestyle=":")

            ax3.axhline(2*sig_dpwv,color="k",linestyle="-.")
            ax3.axhline(-2*sig_dpwv,color="k",linestyle="-.")

            ax3.axhline(3*sig_dpwv,color="k",linestyle="--")
            ax3.axhline(-3*sig_dpwv,color="k",linestyle="--")
                
            # No airmass : add confusion
            #ax_bis.scatter(group["Time"],group["AIRMASS"], color=colors[filt],marker="x",label="airmass")
            p5 = ax3_bis.scatter(df["delta_t_hr"],df["AIRMASS"], color="purple",marker="x",label="airmass")
            p6 = ax3_ter.scatter(df["delta_t_hr"],df["ex_azimuth"], color="darkblue",marker="+",label="azimuth")

            ax3.text(0.05, 0.95, textstr4, transform=ax3.transAxes, fontsize=14,verticalalignment='top', bbox=props)
            
            # bottom right scatter plot dpwv and 
                
            # Normalisation des couleurs
            norm = plt.Normalize(vmin=np.min(dtt), vmax=np.max(dtt))
            cmap = plt.cm.jet
            dtt_colors = cmap(norm(dtt))
                

            # Scatter colore selon dtt
            sc = ax4.scatter(
            daz, dpwv,
            c=dtt,                 # couleur = valeur de dtt
            cmap="jet",        # ou "plasma", "seismic", "turbo" etc.
            marker="o",
            s=100,                  # taille du point
            edgecolors="None"
            )


            # Ajout des barres d'erreur en Y, avec même couleur que le point
            for i in range(len(daz)):
                ax4.errorbar(
                daz[i], dpwv[i],
                yerr=edpwv[i],    # ton array d'erreurs en y
                fmt='None',          # ne redessine pas le point
                ecolor = dtt_colors[i],    # couleur = même que le point
                elinewidth=1.5,
                capsize=2,
                alpha=0.8
                )


            # Ajouter une barre de couleur
            cb = plt.colorbar(sc, ax=ax4, label="Δt (min)")

            ax4.axhline(sig_dpwv,color="k",linestyle = ":",label="$1 \sigma$")
            ax4.axhline(2*sig_dpwv,color="k",linestyle = "-.",label="$2 \sigma$")
            ax4.axhline(3*sig_dpwv,color="k",linestyle = "--",label="$3 \sigma$")
            ax4.set_ylim(0.,dpwv.max()*1.2)
            ax4.legend(loc="upper right")
 
            ax3_bis.set_ylim(2.5,0.5)
            ax3_bis.set_ylabel("airmass")
            ax3_ter.set_ylabel("azimuth")

            ax3_bis.legend(loc="lower right", handles=[p5, p6])
            ax_bis.legend(loc="lower right", handles=[p2, p3])

           

            # Couleur du label de chaque axe
            ax3.yaxis.label.set_color(p4[0].get_color())   # p1 est un tuple (line, caplines, barlinecols)
            ax3_bis.yaxis.label.set_color(p5.get_facecolor()[0])  # scatter -> facecolor
            ax3_ter.yaxis.label.set_color(p6.get_facecolor()[0])

            # Couleur des ticks (graduations) sur chaque axe
            ax3.tick_params(axis='y', colors=p4[0].get_color())
            ax3_bis.tick_params(axis='y', colors=p5.get_facecolor()[0])
            ax3_ter.tick_params(axis='y', colors=p6.get_facecolor()[0])

            # Couleur de la spine (la ligne verticale de l’axe)
            ax3.spines['left'].set_color(p4[0].get_color())
            ax3_bis.spines['right'].set_color(p5.get_facecolor()[0])
            ax3_ter.spines['right'].set_color(p6.get_facecolor()[0])
  
            ax3.axvline(x= 0.0, color='k', linestyle=':', linewidth=2)
            ax3.axhline(y= 0.0, color='r', linestyle='-', linewidth=2)
            ax3.legend(loc="upper right")
            ax3.grid()
            ax3.set_ylabel("$\Delta PWV$ [mm]")
            ax3.set_xlabel("Relative time wrt midnight (hour)")
                #ax3.xaxis.set_major_formatter(date_form)
                #ax3.xaxis_date()
            # MIN,MAX in Delta PWV
            ax3.set_ylim(-2.,2.)
            #ax3.text(0.03, 0.95, textstr, transform=ax3.transAxes, fontsize=14,
            #verticalalignment='top', bbox=props)
            #ax3.text(0.35, 0.95, textstr3, transform=ax3.transAxes, fontsize=14,
            #verticalalignment='top', bbox=props)
                #ax.set_title(f"night {night}")
                #ax.tick_params(axis="x", labelrotation=45)

            ax4.set_title("azimuth distance")
            ax4.set_xlabel("$|\Delta Az|$ (deg)")
            ax4.set_ylabel("$|\Delta PWV|$ (mm)")

            #plt.suptitle(tag,ha="center")
            plt.suptitle(f"night {night}  :  ({the_collection})",ha="center",fontsize=16)
            plt.tight_layout()
            figname =f"{pathfigs}/{prefix}_per_night_{night}"+figtype

            #to save every things including the horizontal colorplot
            plt.savefig(figname,bbox_inches='tight')
            plt.show()

        except Exception as inst:
            print(type(inst))    # the exception type
            print(inst.args)     # arguments stored in .args
            print(inst)  

            
            stat.loc["slope","PWV [mm]_x"] = 0.
            stat.loc["slope_err","PWV [mm]_x"] = 0.
            stat.loc["chi2","PWV [mm]_x"] = -1.

        # add statistics
        all_dateObs_sel[night] = stat
        

## Make a summary of Night quality

In [ ]:
def GetStatistics(all_dateObs_sel):
    df = pd.DataFrame(columns = ["count","mean","median","std","slope","slope_err"])
    
    for nightObs, stat in all_dateObs_sel.items():
        count = int(stat.loc["count"].values[0])
        mean = stat.loc["mean"].values[0]
        median = stat.loc["50%"].values[0]
        std = stat.loc["std"].values[0]
        slope = stat.loc["slope"].values[0]
        slope_err = stat.loc["slope_err"].values[0]
        df.loc[nightObs] = [count,mean,median,std,slope,slope_err]
    return df

In [ ]:
df = GetStatistics(all_dateObs_sel)

In [ ]:
df.head()

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(18,5))
ax1,ax2,ax3  = axs

leg1=ax1.get_legend()
leg2=ax2.get_legend()
leg3=ax3.get_legend()


df["std"].hist(ax=ax1,bins=100,range=(0,2),facecolor="b")
df.plot.scatter(x="median",y="std",ax=ax2,marker="o",color="b",grid=True)
#df.plot.scatter(x="median",y="slope",ax=ax3,marker="o",color="b",grid=True)
ax3.errorbar(
    df["median"],          # x
    df["slope"],           # y
    xerr=df["std"],             # ou df["median_err"] si tu en as
    yerr=df["slope_err"],  # erreur sur y
    fmt="o",               # marqueur
    color="b",             # couleur
    ecolor="gray",         # couleur des barres d’erreur
    capsize=3,             # petites barres aux extrémités
    markersize=8,
)


ax1.set_xlim(0.,1.)
ax1.set_xlabel("$RMS(PWV)$ (mm)")
ax1.set_title("per-night dispersion")

ax2.set_xlabel("$median(PWV)$ (mm)")
ax2.set_ylabel("$RMS(PWV)$ (mm)")
ax2.set_title("per-night dispersion")

ax3.set_ylim(-0.5,0.5)
ax3.set_xlabel("$median(PWV)$ (mm)")
ax3.set_ylabel("$fitted slope dPWV/dt $ (mm/h)")
ax3.set_title("per-night dispersion")
ax3.grid(True)

plt.suptitle(f"PWV in 2025 with OG550 data {the_collection}")

plt.tight_layout()

In [ ]:
def ComputeDataTimeStr(num):
    year = num//10000
    remain = (num-year*10000)
    month = remain//100
    day = remain-100*month
    yearstr = f"{year}"
    monthstr = f"{month}"
    daystr= f"{day}"
    stry = yearstr.zfill(4)
    strm = monthstr.zfill(2)
    strd = daystr.zfill(2)
    
    str = f"{stry}-{strm}-{strd}"
    return str

### Convert dateobs into datetime

In [ ]:
df["Time"] = pd.to_datetime([ ComputeDataTimeStr(num) for  num in df.index])

### Plot summary

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%d")
fig,axs = plt.subplots(2,1,figsize=(18,10),sharex=True)
ax1,ax2  = axs

leg1=ax1.get_legend()
leg2=ax2.get_legend()
        
#df.plot(x="Time",y="median",ax=ax1,marker='+',c="r",lw=0.0,grid=True,ms=10,label=tag,legend=leg1)
df.plot(x="Time",y="median",ax=ax1,marker='+',c="r",lw=0.0,grid=True,ms=10)
ax1.errorbar(x=df["Time"], y=df["median"],yerr=df["std"],fmt=".",color="r",ecolor="k")

ax1.set_ylabel("PWV (mm)")
ax1.set_xlabel("time")
ax1.xaxis.set_major_formatter(date_form)
ax1.set_title(f"Median Precipitable water vapor per night {tag}")
ax1.set_ylim(PWVMIN,PWVMAX)
ax1.legend(loc="upper right")

ax2.set_title("Fitted drift per night")
#ax2.errorbar(x=df["Time"], y=df["slope"],yerr=df["slope_err"],fmt=".",color="r",ecolor="k",ms=10,label=tag,legend=leg2)
ax2.errorbar(x=df["Time"], y=df["slope"],yerr=df["slope_err"],fmt=".",color="r",ecolor="k",ms=10)
ax2.grid()
ax2.set_ylabel("dPWV/dt (mm/h)")
ax2.set_xlabel("time")
ax2.set_ylim(-1.,1.)
ax2.xaxis.set_major_formatter(date_form)
ax2.legend(loc="upper right")

ax2.tick_params(axis='x', labelrotation=45)

plt.tight_layout()
figname =f"{pathfigs}/{prefix}_pwv_medianspredslope_allnights"+figtype
plt.savefig(figname)
plt.show()

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%d")
fig,axs = plt.subplots(1,1,figsize=(18,8))
ax = axs
leg=ax.get_legend()

#df.plot(x="Time",y="median",ax=ax,marker='+',c="r",ms=20,lw=0.0,grid=True,label=tag,legend=leg)
df.plot(x="Time",y="median",ax=ax,marker='+',c="r",ms=20,lw=0.0,grid=True)
ax.errorbar(x=df["Time"], y=df["median"],yerr=df["std"],fmt="o",ms=5,color="r",ecolor="k")
ax.set_ylabel("PWV (mm)")
ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title(f"Median and spread of Precipitable water vapor per night {tag}")
ax.set_ylim(PWVMIN,PWVMAX)
ax.legend(loc="upper right")

if not FLAG_WITHCOLLIMATOR:
    ax.axvspan(TMIN,datetime_WITHCOLLIMATOR, color='yellow', alpha=0.1)

figname =f"{pathfigs}/{prefix}_pwv_medians_allnights"+figtype
plt.savefig(figname)
plt.show()